## 주제

MAP3K5 IC50 활성값 예측 모델 개발

## 데이터

- CAS_KPBMA_MAP3K%_IC50.xlsx
    - CAS에서 수집한 MAP3K5(ASK1) 관련 화합물 리스트
    - Ligand Number: 화합물 번호
    - Substance Name: 물질명
    - SMILDES: 분자구조
    - 등 총 20개 칼럼
- ChEMBL_ASK1(IC50).csv
    - ChEMBl에서 수집된 ASK1 효소 저해 관련 실험 결과 824종
- PubChem_ASK1.csv
    - PubChem에서 수집된 ASK1 저해 활성 정보
- test.csv
    - ID: 고유 ID
    - Smies: 분자구조 데이터
- sample_submission.csv
    - ID: 고유 ID
    - ASK1_IC50_nM: 에측한 IC50(nM단위)

## 코드 흐름

1. 초기 설정 및 라이브러리 로드
    - 기본 라이브러리: pandas, numpy 등 데이터 처리 라이브러리 임포트
    - 화학 정보학 라이브러리: rdkit, chenprop
    - 모델 라이브러리: catboost
    - 시스템 설정: random.seed, numpy,random.seed, torch.manual_seed을 통해 실험의 재현성 확보
2. 데이터 로드 및 분자 특성 추출
    - 데이터 로드: train.csv와 test.csv 파일을 불러와 SMILES(분자 구조 문자열)와 타겟값(pIC50)을 준비한다.
    - RDKIT Descriptors 추출:
        - MolFromSmiles, AllChem.GetMorgonFingerprint 등 RDKIT 함수를 사용하여 분자의 다양한 물리/화학적 특성을 추출한다. 이들은 트리 모델의 입력 피처로 사용된다.
3. D-MPNN 임베딩 추출(GNN 활용)
    - Chemprop 모델 로듸 D-MPNN 구조를 가진 모델을 로드하거나 학습시킨다.
    - 임베딩 생성: SMILES 문자열을 D-MPNN 모델에 입력하여 분자의 구조적 정보를 압축한 고차원의 분자 임베딩 벡터를 추출한다.
    - 목적: D-MPNN 임베딩은 RDKIT 특성만으로는 담을 수 없는 복잡한 분자 연결성과 위상적 정보를 담고 있다.
4. 데이터 증강
    - SMILES Enumeration:
        - 하나의 분자로부터 여러 개의 유효한 SMILES 표현을 생성하여 데이터셋의 크기와 다양성을 늘린다. 이는 모델이 분자 표기법의 변화에 덜 민감하도록 일반화를 돕는다.
    - Mixup:
        - 두 데이터 샘플을 선형 결합하여 새로운 데이터를 생성하는 기법을 적용한다. 특히, 고활성 분자 데이터를 증강하는 데 집중하여, 희소한 고활성 영역에서의 예측 성능을 높인다.
5. CatBoost 학습 및 커스텀 손실 함수
    - 데이터 결합: D-MPNN 임베딩 벡터와 RDKIT 디스크립터를 하나로 합쳐서 최종 학습 피처(X)를 만든다.
    - 커스텀 손실 함수 정의:
        - CatBoost에 사용될 사용자 정의 손실 함수 정의
        - 이 함수는 고활성 타겟 값에 높은 가중치를 부여하여 해당 영역의 예측 오차를 적극적으로 줄이도록 모델을 유도한다.
    - CatBoost 모델 학습: 정의된 커스텀 손실 함수를 목적 함수로 사용하여 최종 CatBoost 모델을 학습시킨다.
6. 최종 예측 및 제출
    - 테스트 데이터 준비: 테스트 데이터에 대해서도 동일하게 D-MPNN 임베딩과 RDKIT 특성을 추출하고 결합한다.
    - 에측: 학습된 CatBoost 모델을 사용하여 테스트 데이터의 pIC50을 예측한다.
    - 파일 저장: 예측 결과를 submission.csv 형식으로 저장

## 차별점 및 배울점

- 단 하나의 피처 타입에 의존하지 않고 딥러닝과 전통적인 화학 정보를 결합한 점이 흥미로웠다.
- 데이터셋의 근본적인 문제(적은 데이터, 고활성 분자 희소성)를 증강 기법, Mixup을 통해 해결하였다.
- 단순히 표준 손실 함수를 쓰는 것을 넘어 대회 규칙에 맞추어서 모델을 학습시켰다.
- 딥러닝과 머신러닝을 결합하여 예측력을 높였다.